In [17]:
from pathlib import Path
import pandas as pd
import altair as alt
alt.data_transformers.enable('data_server')

DataTransformerRegistry.enable('data_server')

In [18]:
Path.cwd()

PosixPath('/Users/ben/Developer/SpaceMissionDES/SpaceMissionDES/analysis')

#### Load the Data

In [19]:
data_dir = Path.cwd().parent.parent / "results" / "test-05"

mc_results = pd.read_csv(data_dir / "mc.csv")
mc_results.head()

,replicant,outcome,duration,anomaly_count,anomaly_time,anomaly_vehicle,anomaly_activity
0,2,False,4.0,2,4.0,SEP-02 + EOI-MCPS,Ascent
1,1,False,3111.0,2,3111.0,MOI-MCPS,Lunar Insertion
2,0,False,3116.0,2,3100.0,HAB,Countdown
3,3,True,3450.0,0,NaN,NaN,NaN
4,7,False,1.0,2,1.0,SEP-02 + EOI-MCPS,Countdown


#### Summary Statistics

In [20]:
N = mc_results.replicant.max()
success_rate = sum(mc_results.groupby("replicant").first().outcome) / N
print(f"The success rate was: {success_rate} = {100*success_rate:.3f} %\n")

The success rate was: 0.5735735735735735 = 57.357 %



#### Mission Duration

In [21]:
df = (
    # mc_results[mc_results.outcome == True]
    mc_results
    .groupby("replicant", as_index=False)
    .first()
    .loc[:, ["replicant", "outcome", "duration"]]
)

df.loc[df.outcome == False, "duration"] = 1e6

alt.Chart(df).mark_bar().encode(
    # alt.X("duration:Q", bin=True),
    alt.X("duration:Q", bin=alt.Bin(extent=[50, 200], step=10)),
    y='count()',
).properties(
    width = 600
)

alt.Chart(...)

Compute the Emprical CDF

In [22]:
import numpy as np

# from: https://stackoverflow.com/questions/33345780/empirical-cdf-in-python-similiar-to-matlabs-one
def ecdf(sample):

    # convert sample to a numpy array, if it isn't already
    sample = np.atleast_1d(sample)

    # find the unique values and their corresponding counts
    quantiles, counts = np.unique(sample, return_counts=True)

    # take the cumulative sum of the counts and divide by the sample size to
    # get the cumulative probabilities between 0 and 1
    cumprob = np.cumsum(counts).astype(np.double) / sample.size

    return quantiles, cumprob

def plot_ECDF(df):
    sample = df.duration.values
    qe, pe = ecdf(sample)
    data = pd.DataFrame({"duration": list(qe), "prob": list(pe)})
    fig = alt.Chart(data).mark_line().encode(
        x = alt.X("duration", scale=alt.Scale(domain=(0,200))),
        y = "prob"
    ).properties(width=500, height=250)
    return fig


In [23]:
fig = plot_ECDF(df)
fig.interactive()

alt.Chart(...)

In [24]:
fig = plot_ECDF(df)
fig.interactive()

alt.Chart(...)

In [25]:
df = (
    # mc_results[mc_results.outcome == True]
    mc_results
    .groupby("replicant", as_index=False)
    .first()
    .loc[:, ["replicant", "outcome", "duration"]]
)

df.loc[df.outcome == False, "duration"] = 1e6

df

,replicant,outcome,duration
0,0,False,1000000.0
1,1,False,1000000.0
2,2,False,1000000.0
3,3,True,3450.0
4,4,True,3450.0
...,...,...,...
995,995,True,3450.0
996,996,True,3450.0
997,997,True,3450.0
998,998,False,1000000.0


In [26]:
df

sample = df.duration.values
sample

# convert sample to a numpy array, if it isn't already
sample = np.atleast_1d(sample)

# find the unique values and their corresponding counts
quantiles, counts = np.unique(sample, return_counts=True)

# take the cumulative sum of the counts and divide by the sample size to
# get the cumulative probabilities between 0 and 1
cumprob = np.cumsum(counts).astype(np.double) / sample.size

#### Failure Diagnostics (Aggregate)

Which Vehicles Fail

In [27]:
# Relative failure frequency by vehicle
df = (
    mc_results
    .groupby("anomaly_vehicle")
    .count()
    .rename(columns={'replicant': 'n'})
    .loc[:,'n']
)

fail_count = df.sum()
# (df / fail_count).reset_index()

alt.Chart(
    (df / fail_count).reset_index()
).mark_bar().encode(
    y = "anomaly_vehicle",
    x = "n"
)

alt.Chart(...)

Which Activities Fail?

In [28]:
# Relative failure frequency by activity
df = (
    mc_results
    .groupby("anomaly_activity")
    .count()
    .rename(columns={'replicant': 'n'})
    .loc[:,'n']
)

fail_count = df.sum()
# (df / fail_count).reset_index()

alt.Chart(
    (df / fail_count).reset_index()
).mark_bar().encode(
    y = "anomaly_activity",
    x = "n"
)

alt.Chart(...)

#### Failure Diagnostics (per mission)

Avg ascent failures per mission

In [29]:
mc_results.head()

,replicant,outcome,duration,anomaly_count,anomaly_time,anomaly_vehicle,anomaly_activity
0,2,False,4.0,2,4.0,SEP-02 + EOI-MCPS,Ascent
1,1,False,3111.0,2,3111.0,MOI-MCPS,Lunar Insertion
2,0,False,3116.0,2,3100.0,HAB,Countdown
3,3,True,3450.0,0,NaN,NaN,NaN
4,7,False,1.0,2,1.0,SEP-02 + EOI-MCPS,Countdown


In [30]:
mc_results.query("anomaly_count > 1")
# df = mc_results.query("replicant == 3")
# df

,replicant,outcome,duration,anomaly_count,anomaly_time,anomaly_vehicle,anomaly_activity
0,2,False,4.0,2,4.0,SEP-02 + EOI-MCPS,Ascent
1,1,False,3111.0,2,3111.0,MOI-MCPS,Lunar Insertion
2,0,False,3116.0,2,3100.0,HAB,Countdown
4,7,False,1.0,2,1.0,SEP-02 + EOI-MCPS,Countdown
8,8,False,3116.0,2,3100.0,HAB,Countdown
...,...,...,...,...,...,...,...
991,985,False,3103.0,2,3103.0,Orion,Countdown
992,986,False,1.0,2,1.0,SEP-02 + EOI-MCPS,Countdown
994,988,False,3102.0,2,3102.0,TMI-MCPS,Countdown
995,989,False,1.0,2,1.0,SEP-01 + TEI-MCPS,Countdown


In [31]:
# df.anomaly_activity.value_counts()

In [32]:
df = mc_results.groupby("replicant", as_index=False)["anomaly_activity"].value_counts()
df

,replicant,anomaly_activity,count
0,0,Countdown,1
1,1,Lunar Insertion,1
2,2,Ascent,1
3,7,Countdown,1
4,8,Countdown,1
...,...,...,...
426,985,Countdown,1
427,986,Countdown,1
428,988,Countdown,1
429,989,Countdown,1


In [33]:
# alt.Chart(df.unstack().Ascent.reset_index()).mark_boxplot().encode(x = "Ascent")

In [34]:
df = (
    mc_results
    # mc_results[mc_results.anomaly_activity != "Countdown"]
    .groupby("replicant", as_index=False)
    ["anomaly_activity"]
    .value_counts()
)

alt.Chart(df).mark_boxplot().encode(
    x = "count",
    y = "anomaly_activity"

)

alt.Chart(...)

In [35]:
df[df.anomaly_activity != "Countdown"]

,replicant,anomaly_activity,count
1,1,Lunar Insertion,1
2,2,Ascent,1
5,10,Crew Landing on Earth,1
10,20,Ascent,1
11,21,Lunar Transit,1
...,...,...,...
418,971,Ascent,1
419,974,Lunar Insertion,1
420,976,Mars Transit,1
422,978,Lunar Insertion,1


In [36]:
mc_results

,replicant,outcome,duration,anomaly_count,anomaly_time,anomaly_vehicle,anomaly_activity
0,2,False,4.0,2,4.0,SEP-02 + EOI-MCPS,Ascent
1,1,False,3111.0,2,3111.0,MOI-MCPS,Lunar Insertion
2,0,False,3116.0,2,3100.0,HAB,Countdown
3,3,True,3450.0,0,NaN,NaN,NaN
4,7,False,1.0,2,1.0,SEP-02 + EOI-MCPS,Countdown
...,...,...,...,...,...,...,...
1004,995,True,3450.0,0,NaN,NaN,NaN
1005,996,True,3450.0,0,NaN,NaN,NaN
1006,998,False,3101.0,2,3101.0,MOI-MCPS,Countdown
1007,999,True,3450.0,0,NaN,NaN,NaN


---
### Experiments

In [37]:
import pandas as pd
import altair as alt
import numpy as np

# from: https://stackoverflow.com/questions/33345780/empirical-cdf-in-python-similiar-to-matlabs-one
def ecdf(sample):

    # convert sample to a numpy array, if it isn't already
    sample = np.atleast_1d(sample)

    # find the unique values and their corresponding counts
    quantiles, counts = np.unique(sample, return_counts=True)

    # take the cumulative sum of the counts and divide by the sample size to
    # get the cumulative probabilities between 0 and 1
    cumprob = np.cumsum(counts).astype(np.double) / sample.size

    return quantiles, cumprob

sample = [1, 2, 2, 3, 3, 3, 4, 4, 5, 5,]
qe, pe = ecdf(sample)
data = pd.DataFrame({"duration": list(qe), "prob": list(pe)})

alt.Chart(data).mark_line(interpolate='step-after', point=True, color="red").encode(
    x = alt.X("duration", axis=alt.Axis(values=[0, 1, 2, 3, 4, 5]), scale=alt.Scale(domain=(1,5))),
    y = "prob"
).properties(
    width=400, 
    height=400
)

alt.Chart(...)